In [1]:
!pip install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/58/b8/51b956c2da9729390a3080397cd2f31171394543af7746681466e372f69a/torch-2.2.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/b5/56/38e892200f8638032b64f6fc8660049f0d00ccba086cf1dcb884bd6370d2/torchvision-0.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/07/1f/ddc210fc946855233d9be29d6509fb0c06803416b500a6ae7414f2371edb/torchaudio-2.2.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/b7/f4/6a90020cd2d93349b442bfcb657d0dc91eee65491600b2cb1d388bc98e6b/typing_extensions-4.9.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/198.6 MB 1.9 MB/s et

In [2]:
!pip install torch torchvision torchaudio

In [3]:
!pip install --upgrade typing-extensions

In [4]:
!pip install python3.10

ERROR: Could not find a version that satisfies the requirement python3.10 (from versions: none)
ERROR: No matching distribution found for python3.10


In [5]:
import torch
print(torch.__version__)

2.2.0+cpu


In [6]:
!pip install transformers

In [7]:
## Preliminaries
"""
We define a function to normalize a tweet to the format we used for TweetEval. Note that preprocessing is minimal (replacing user names by `@user` and links by `http`).
"""

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [8]:
from transformers import pipeline, AutoTokenizer, AutoModel, TFAutoModel
import numpy as np
from scipy.spatial.distance import cosine
from collections import defaultdict
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from scipy.special import softmax
import csv
import urllib.request

C:\Users\SUNYLoaner\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [9]:
MODEL = "cardiffnlp/twitter-roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL)

C:\Users\SUNYLoaner\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SUNYLoaner\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
def get_embedding(text):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt')
  features = model(**encoded_input)
  features = features[0].detach().cpu().numpy()
  features_mean = np.mean(features[0], axis=0)
  return features_mean

In [11]:
query = "The book was awesome"

tweets = ["I just ordered fried chicken 🐣",
          "The movie was great",
          "What time is the next game?",
          "Just finished reading 'Embeddings in NLP'"]

d = defaultdict(int)
for tweet in tweets:
  sim = 1-cosine(get_embedding(query),get_embedding(tweet))
  d[tweet] = sim

print('Most similar to: ',query)
print('----------------------------------------')
for idx,x in enumerate(sorted(d.items(), key=lambda x:x[1], reverse=True)):
  print(idx+1,x[0])


Most similar to:  The book was awesome
----------------------------------------
1 The movie was great
2 Just finished reading 'Embeddings in NLP'
3 I just ordered fried chicken 🐣
4 What time is the next game?


In [12]:
MODEL = "cardiffnlp/twitter-roberta-base"
text = "Good night 😊"
text = preprocess(text)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Pytorch
encoded_input = tokenizer(text, return_tensors='pt')
model = AutoModel.from_pretrained(MODEL)
features = model(**encoded_input)
features = features[0].detach().cpu().numpy()
features_mean = np.mean(features[0], axis=0)


features_mean.shape

(768,)

In [13]:
MODEL = "cardiffnlp/twitter-roberta-base"
fill_mask = pipeline("fill-mask", model=MODEL, tokenizer=MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

def print_candidates():
    for i in range(5):
        token = tokenizer.decode(candidates[i]['token'])
        score = np.round(candidates[i]['score'], 4)
        print(f"{i+1}) {token} {score}")

texts = [
 "I am so <mask> 🤦🏻‍♂",
 "I am so <mask> 🥳",
"I am so <mask> 🍻"
]
for text in texts:
    t = preprocess(text)
    print(f"{'-'*30}\n{t}")
    candidates = fill_mask(t)
    print_candidates()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


------------------------------
I am so <mask> 🤦🏻‍♂
1)  tired 0.0814
2)  confused 0.0713
3)  hungry 0.0482
4)  mad 0.0463
5)  weak 0.0354
------------------------------
I am so <mask> 🥳
1)  happy 0.3843
2)  excited 0.2066
3)  proud 0.1178
4)  blessed 0.0427
5)  grateful 0.0258
------------------------------
I am so <mask> 🍻
1)  hungry 0.2078
2)  happy 0.1753
3)  excited 0.1415
4)  full 0.0385
5)  proud 0.036


In [14]:
task='emotion'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

text = "Why can't I find a good Indian restaurant"
print(text)
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

Why can't I find a good Indian restaurant
1) anger 0.5733
2) sadness 0.398
3) joy 0.0151
4) optimism 0.0135


In [15]:
text = "you are pretty good"
print(text)
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

you are pretty good
1) joy 0.8186
2) optimism 0.115
3) anger 0.0425
4) sadness 0.0239


In [16]:
from sklearn.metrics import classification_report
import argparse
import os
import sys 

TASKS = [
    'emoji', 
    'emotion',
    'hate',
    'irony',
    'offensive',
    'sentiment',
    'stance']

STANCE_TASKS = [
    'abortion',
    'atheism',
    'climate',
    'feminist',
    'hillary']

def load_gold_pred(args):
    tweeteval_path = args.tweeteval_path
    predictions_path = args.predictions_path
    task = args.task

    if 'stance' in task:
        gold = []
        pred = []
        for stance_t in STANCE_TASKS:
            gold_path = os.path.join(tweeteval_path,task,stance_t,'test_labels.txt')
            pred_path = os.path.join(predictions_path,task,stance_t+'.txt')
            gold.append(open(gold_path).read().split("\n")[:-1])
            pred.append(open(pred_path).read().split("\n")[:-1])
        # flatten lists of lists
        gold = [p for each_target in gold for p in each_target]
        pred = [p for each_target in pred for p in each_target]
    else:
        gold_path = os.path.join(tweeteval_path,task,'test_labels.txt')
        pred_path = os.path.join(predictions_path,task+'.txt')
        gold = open(gold_path).read().split("\n")[:-1]
        pred = open(pred_path).read().split("\n")[:-1]
        
    return gold, pred

def single_task_results(args):
    task = args.task
    tweeteval_result = -1
    results = {}
    
    try:
        gold, pred = load_gold_pred(args)
        results = classification_report(gold, pred, output_dict=True)

#         # Emoji (Macro f1)
        if 'emoji' in task:
            tweeteval_result = results['macro avg']['f1-score'] 

        # Emotion (Macro f1)
        elif 'emotion' in task:
            tweeteval_result = results['macro avg']['f1-score'] 

        # Hate (Macro f1)
        elif 'hate' in task:
            tweeteval_result = results['macro avg']['f1-score'] 

        # Irony (Irony class f1)
        elif 'irony' in task:
            tweeteval_result = results['1']['f1-score'] 

        # Offensive (Macro f1)
        elif 'offensive' in task:
            tweeteval_result = results['macro avg']['f1-score'] 

        # Sentiment (Macro Recall)
        elif 'sentiment' in task:
            tweeteval_result = results['macro avg']['recall']

        # Stance (Macro F1 of 'favor' and 'against' classes)
        elif 'stance' in task:
            f1_against = results['1']['f1-score']
            f1_favor = results['2']['f1-score']
            tweeteval_result = (f1_against+f1_favor) / 2
            
    except Exception as ex:
        print(f"Issues with task {task}: {ex}")
        
    return tweeteval_result, results

def is_all_good(all_tweeteval_results):
    return all([r != -1 for r in all_tweeteval_results.values()])


if __name__=="__main__":
    parser = argparse.ArgumentParser(description='TweetEval evaluation script.')
    
    parser.add_argument('--tweeteval_path', default="./datasets/", type=str, help='Path to TweetEval datasets')
    parser.add_argument('--predictions_path', default="./predictions/", type=str, help='Path to predictions files')
    parser.add_argument('--task', default="", type=str, help='Indicate this parameter to get single task detailed results')

    # Modify this line to handle Jupyter Notebook environment
    args = parser.parse_args([] if 'ipykernel' in sys.argv[0] else None)

    if args.task == "":
        all_tweeteval_results = {}
        
        # Results for each task
        for t in TASKS:
            args.task = t
            all_tweeteval_results[t], _ = single_task_results(args)
            
        # Print results (score=-1 if some results are missing)
        print(f"{'-'*30}")
        if is_all_good(all_tweeteval_results):
            tweeteval_final_score = sum(all_tweeteval_results.values())/len(all_tweeteval_results.values())
        else:
            tweeteval_final_score = -1
        print("aggregation of the individual task scores(F-1 Score):")
        for t in TASKS:
            # Each score
            print(f"{t}: {all_tweeteval_results[t]}") 
        # Final score
        print(f"{'-'*30}\nTweetEval Score: {tweeteval_final_score}")
        
    else:
        # Detailed results of one single task (--task parameter)
        tweeteval_resut, results = single_task_results(args)
        for k in results:
            print(k, results[k])
        print(f"{'-'*30}\nTweetEval Score ({args.task}): {tweeteval_resut}")



------------------------------
aggregation of the individual task scores(F-1 Score):
emoji: 0.3155243507716184
emotion: 0.7982724123055319
hate: 0.5547114323640363
irony: 0.6247755834829443
offensive: 0.8155092112424851
sentiment: 0.7285672376332831
stance: 0.7243628109019552
------------------------------
TweetEval Score: 0.6516747198145507
